In [1]:
!pip install --force-reinstall --no-cache-dir tenacity==8.2.3 --user
!pip install "ibm-watsonx-ai==1.0.8" --user
!pip install "ibm-watson-machine-learning==1.0.367" --user
!pip install "langchain-ibm==0.1.7" --user
!pip install "langchain-community==0.2.10" --user
!pip install "langchain-experimental==0.0.62" --user
!pip install "langchainhub==0.1.18" --user
!pip install "langchain==0.2.11" --user
!pip install "pypdf==4.2.0" --user
!pip install "chromadb==0.4.24" --user

  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.2.3
    Uninstalling tenacity-8.2.3:
      Successfully uninstalled tenacity-8.2.3
  Using cached chromadb-0.4.24-py3-none-any.whl.metadata (7.3 kB)
Using cached chromadb-0.4.24-py3-none-any.whl (525 kB)


In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

In [3]:
# Import the PyPDFLoader class from langchain_community's document_loaders module
# This loader is specifically designed to load and parse PDF files
from langchain_community.document_loaders import PyPDFLoader

# Create a PyPDFLoader instance by passing the URL of the PDF file
# The loader will download the PDF from the specified URL and prepare it for loading
loader = PyPDFLoader("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf")

# Call the load() method to:
# 1. Download the PDF if needed
# 2. Extract text from each page
# 3. Create a list of Document objects, one for each page of the PDF
# Each Document will contain the text content of a page and metadata including page number
document = loader.load()

In [4]:
print(document[1].page_content[:100])  # print the page 1's first 100 tokens

LangChain helps us to unlock the ability to harness the 
LLM’s immense potential in tasks such as do


In [5]:
# Import the CharacterTextSplitter class from langchain.text_splitter module
# Text splitters are used to divide large texts into smaller, manageable chunks
from langchain.text_splitter import CharacterTextSplitter

# Create a CharacterTextSplitter with specific configuration:
# - chunk_size=200: Each chunk will contain approximately 200 characters
# - chunk_overlap=20: Consecutive chunks will overlap by 20 characters to maintain context
# - separator="\n": Text will be split at newline characters when possible
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20, separator="\n")

# Split the previously loaded document (PDF or other text) into chunks
# The split_documents method:
# 1. Takes a list of Document objects
# 2. Splits each document's content based on the configured parameters
# 3. Returns a new list of Document objects where each contains a chunk of text
# 4. Preserves the original metadata for each chunk
chunks = text_splitter.split_documents(document)

# Print the total number of chunks created
# This shows how many smaller Document objects were generated from the original document(s)
# The number depends on the original document length and the chunk_size setting
print(len(chunks))

148


In [6]:
chunks[5].page_content   # take a look at any chunk's page content

'contextualized language models to introduce MindGuide, an \ninnovative chatbot serving as a mental health assistant for \nindividuals seeking guidance and support in these critical areas.'

In [7]:
# Import the EmbedTextParamsMetaNames class from ibm_watsonx_ai.metanames module
# This class provides constants for configuring Watson embedding parameters
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames

# Configure embedding parameters using a dictionary:
# - TRUNCATE_INPUT_TOKENS: Limit the input to 3 tokens (very short, possibly for testing)
# - RETURN_OPTIONS: Request that the original input text be returned along with embeddings
embed_params = {
 EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
 EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

In [8]:
# Import the WatsonxEmbeddings class from langchain_ibm module
# This provides an integration between LangChain and IBM's Watson AI services
from langchain_ibm import WatsonxEmbeddings

# Create a WatsonxEmbeddings instance with the following configuration:
# - model_id: Specifies the "slate-125m-english-rtrvr" embedding model from IBM
# - url: The endpoint URL for the Watson service in the US South region
# - project_id: The Watson project ID to use ("skills-network")
# - params: The embedding parameters configured earlier
watsonx_embedding = WatsonxEmbeddings(
    model_id="ibm/slate-125m-english-rtrvr",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="skills-network",
    params=embed_params,
)

In [9]:
texts = [text.page_content for text in chunks]

embedding_result = watsonx_embedding.embed_documents(texts)
embedding_result[0][:5]

[-0.035563346, -0.012706485, -0.019341178, -0.04773982, -0.018180432]

In [10]:
from langchain.vectorstores import Chroma

In [11]:
!pip install posthog==2.4.2

In [12]:
import os
os.environ["DISABLE_TELEMETRY"] = "true"

In [13]:
!pip install --upgrade chromadb

  Using cached chromadb-1.0.16-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
Using cached chromadb-1.0.16-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.6 MB)
  Attempting uninstall: chromadb
    Found existing installation: chromadb 0.4.24
    Uninstalling chromadb-0.4.24:
      Successfully uninstalled chromadb-0.4.24


In [14]:
docsearch = Chroma.from_documents(chunks, watsonx_embedding)

In [15]:
query = "Langchain"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

LangChain helps us to unlock the ability to harness the 
LLM’s immense potential in tasks such as document analysis, 
chatbot development, code analysis, and countless other


In [16]:
from langchain.chains import RetrievalQA

In [17]:
model_id = 'meta-llama/llama-3-3-70b-instruct' 

parameters = {
    GenParams.MAX_NEW_TOKENS: 256,  # this controls the maximum number of tokens in the generated output
    GenParams.TEMPERATURE: 0.2, # this randomness or creativity of the model's responses 
}

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com"
    # "api_key": "your api key here"
    # uncomment above and fill in the API key when running locally
}

project_id = "skills-network"

model = ModelInference(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)

In [18]:
llama_llm = WatsonxLLM(model = model)

In [19]:
# Create a RetrievalQA chain by configuring:
qa = RetrievalQA.from_chain_type(
    # The language model to use for generating answers
    llm=llama_llm,
    
    # The chain type "stuff" means all retrieved documents are simply concatenated and passed to the LLM
    chain_type="stuff",
    
    # The retriever component that will fetch relevant documents
    # docsearch.as_retriever() converts the vector store into a retriever interface
    retriever=docsearch.as_retriever(),
    
    # Whether to include the source documents in the response
    # Set to False to return only the generated answer
    return_source_documents=False
)

# Define a query to test the QA system
# This question asks about the main topic of the paper
query = "what is this paper discussing?"

# Execute the QA chain with the query
# This will:
# 1. Send the query to the retriever to get relevant documents
# 2. Combine those documents using the "stuff" method
# 3. Send the query and combined documents to the Llama LLM
# 4. Return the generated answer (without source documents)
qa.invoke(query)

{'query': 'what is this paper discussing?',
 'result': " I don't know. \n(Explanation: The text appears to be a snippet from a research paper or academic article, but it doesn't provide enough context to determine the specific topic or subject being discussed. While it mentions a chatbot and LangChain, it doesn't give enough information to understand the main focus of the paper.)"}